In [2]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv

In [3]:
scaled_events = []
for direction in ['reverse', 'template']:
    for i in range(1,9):
        sequence = f'{direction}_{i}'
        scaled_events.append((sequence, pd.read_csv(f'./5_aligned_events/events_{sequence}_scaled_id_collapse.tsv', delimiter='\t')))

In [4]:
event_level_means = np.full((1000, 174, 16), np.nan)
# event_means is a 3D matrix, where the (i,j,k) entry contains the event_level mean of the i^th read, j^th kmer, k^th reference sequence
for j in range(16):
    df = scaled_events[j][1]
    df = df[df['event_stdv'] < 5]
    grouped = df.groupby('read_name')
    read_num = 0
    for read_id, group_data in grouped:
        if read_num >= 1000:
            break
        event_mean_vec = group_data['event_level_mean'].values
        positions_vec = group_data['position'].values
        event_level_means[read_num, positions_vec, j] = event_mean_vec
        read_num += 1

In [10]:
# We look at each spacer, and find the mean and std of current level for that particular spacer. 
# This shows that different spacers have different mean current levels which cannot be explained by randomness alone
# All of this is done in reverse sequence 1
spacer_RC = 'GCATCATTTCC'
spacer_9mer_mid = 'GCATCATTT'
spacer_pos_mid = np.array([34, 55, 76, 97, 118])
for pos in spacer_pos_mid:
    pos_mean = np.nanmean(event_level_means[:,pos,0])
    pos_std = np.nanstd(event_level_means[:,pos,0])
    pos_valid_count = np.count_nonzero(~np.isnan(event_level_means[:,pos,0]))
    print(f'Spacer at position {pos}: valid_reads = {pos_valid_count} mean_current = {pos_mean:.4f}, std = {pos_std:.4f}, sample_mean_std = {pos_std/np.sqrt(pos_valid_count):.4f}')

Spacer at position 34: valid_reads = 209 mean_current = 113.0038, std = 3.2988, sample_mean_std = 0.2282
Spacer at position 55: valid_reads = 207 mean_current = 120.9339, std = 2.8447, sample_mean_std = 0.1977
Spacer at position 76: valid_reads = 214 mean_current = 119.5914, std = 3.3442, sample_mean_std = 0.2286
Spacer at position 97: valid_reads = 213 mean_current = 121.1734, std = 2.5469, sample_mean_std = 0.1745
Spacer at position 118: valid_reads = 215 mean_current = 120.8987, std = 2.7284, sample_mean_std = 0.1861


- In the above code, we have looked for the spacer 9mer CATCATTTC in the reverse 1 sequence. It occurs 5 times, in 5 different 'environments' within the reverse_1 sequence. An environment of a 9mer is the bases adjacent/near to that 9mer. The hypothesis is that the environment of a 9mer has an impact on the observed current level, so that a 9mer current table cannot fully describe the nanopore process. 
- We have found that the first occurrence of this has significantly different current levels compared to the other occurences. 
- The remaining four occurences have very similar current levels, but are still distinct, and in general their confidence intervals do not overlap.

In [6]:
# We look at a particular spacer, and find the means and std across each sequence
spacer_RC = 'GCATCATTTCC'
spacer_9mer_mid = 'GCATCATTT'
pos = 76
for j in range(8):
    pos_mean = np.nanmean(event_level_means[:,pos,j])
    pos_std = np.nanstd(event_level_means[:,pos,j])
    pos_valid_count = np.count_nonzero(~np.isnan(event_level_means[:,pos,j]))
    print(f'Spacer at position {pos} in reverse_{j+1}: valid_reads = {pos_valid_count} mean_current = {pos_mean:.4f}, std = {pos_std:.4f}, sample_mean_std = {pos_std/np.sqrt(pos_valid_count):.4f}')

Spacer at position 76 in reverse_1: valid_reads = 768 mean_current = 120.8094, std = 2.9020, sample_mean_std = 0.1047
Spacer at position 76 in reverse_2: valid_reads = 123 mean_current = 120.2189, std = 2.3780, sample_mean_std = 0.2144
Spacer at position 76 in reverse_3: valid_reads = 25 mean_current = 120.2684, std = 3.5593, sample_mean_std = 0.7119
Spacer at position 76 in reverse_4: valid_reads = 214 mean_current = 119.5914, std = 3.3442, sample_mean_std = 0.2286
Spacer at position 76 in reverse_5: valid_reads = 78 mean_current = 121.4756, std = 2.3056, sample_mean_std = 0.2611
Spacer at position 76 in reverse_6: valid_reads = 790 mean_current = 120.2349, std = 2.3312, sample_mean_std = 0.0829
Spacer at position 76 in reverse_7: valid_reads = 808 mean_current = 121.2752, std = 2.3417, sample_mean_std = 0.0824
Spacer at position 76 in reverse_8: valid_reads = 748 mean_current = 119.6906, std = 2.4000, sample_mean_std = 0.0878


- In the above code, we again look at the same 9mer spacer in different environments - but this time the different environments are across different sequences entirely. 
- We repeatedly look at the 3rd spacer. There is some impact from the environment, but this is not major.
- reverse_7 and reverse_8 current levels are separated by 0.22pA, which is about 4 standard deviations in the sample mean sense.

In [7]:
# We look at a particular spacer, and find the means and std across all sequences
spacer_RC = 'GCATCATTTCC'
spacer_9mer_mid = 'GCATCATTT'
pos = 76

pos_mean = np.nanmean(event_level_means[:,pos,[0,5,6,7]])
pos_std = np.nanstd(event_level_means[:,pos,0:8])
pos_valid_count = np.count_nonzero(~np.isnan(event_level_means[:,pos,0:8]))
print(f'Spacer at position {pos}: valid_reads = {pos_valid_count} mean_current = {pos_mean:.4f}, std = {pos_std:.4f}, sample_mean_std = {pos_std/np.sqrt(pos_valid_count):.4f}')

Spacer at position 76: valid_reads = 3554 mean_current = 120.5158, std = 2.6354, sample_mean_std = 0.0442


- The above code combines all the current levels (across different sequences) corresponding to the spacer at position 77 (3rd spacer).
- The hypothesis is that because different environments have been combined in this case, the standard deviation should be higher than for the separated environments.
- This is a mathematical fact. If $X$ is a mixture distribution of $X_1, X_2, \ldots, X_k$, with probabilities $p_1, p_2, \ldots, p_k$, then 
$$\mathbb{E}[X]=\sum_{i=1}^k p_i \mathbb{E}[X_i]$$
$$Var[X] = \sum_{i=1}^k p_i Var[X_i] + \sum_{i=1}^k p_i \mathbb{E}[X_i]^2-\left(\sum_{i=1}^k p_i \mathbb{E}[X_i]\right)^2\ge \sum_{i=1}^k p_i Var[X_i]

In [8]:
plt.scatter(r1_pos2['event_level_mean'], r1_pos2['event_stdv'], marker='.', alpha=0.3);
plt.title('event_stdv vs event_level_mean for TGTCGGATT')
plt.xlabel('event_level_mean (pA)')
plt.ylabel('event_stdv (pA)');

NameError: name 'r1_pos2' is not defined

This plot shows a surprising relationship. We are looking at the 9mer TGTCGGATT across all reads. In reads where the event_level_mean is higher, we see that event_stdv is also higher. These uncertain reads were previously skewing our mean, but the second weighted method adjusts for this. 

In [ ]:
# Temp

array([3, 3, 3, 4])